# Testing Algorithms on Mushroom Dataset
* Dataset: https://archive.ics.uci.edu/ml/datasets/Mushroom

In [16]:
import pandas as pd 
import numpy as np 

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import StratifiedKFold

In [2]:
#importing data
data = pd.read_csv('datasets/mushroom.csv',header=0,names=['edible','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above','stalk-surface-below','stalk-color-above','stalk-color-below','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat'])

In [3]:
data.head()

,edible,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below,stalk-color-above,stalk-color-below,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
1,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
2,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
3,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
4,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g


In [5]:
#storing the label 
mushroom_label = data['edible']

#removing label from dataframe 
data.drop(['edible'],axis=1,inplace=True)

In [6]:
#one hot encoding
data = pd.get_dummies(data)

In [9]:
data.drop(['stalk-root_?'],axis=1,inplace=True)

In [10]:
data.head()

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [12]:
#creating target array and features array 
Y = mushroom_label.replace({"e":1, "p":0})
X = data

In [15]:
#creating 5 sets of training and test data 
X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y, train_size=5000)
X2_train, X2_test, y2_train, y2_test = train_test_split(X, Y, train_size=5000)
X3_train, X3_test, y3_train, y3_test = train_test_split(X, Y, train_size=5000)
X4_train, X4_test, y4_train, y4_test = train_test_split(X, Y, train_size=5000)
X5_train, X5_test, y5_train, y5_test = train_test_split(X, Y, train_size=5000)

In [18]:
#creating arrays of training and test sets 
train_X_sets = [X1_train,X2_train,X3_train,X4_train,X5_train]
test_X_sets = [X1_test,X2_test,X3_test,X4_test,X5_test]

#creating arrays of training and test sets 
train_y_sets = [y1_train,y2_train,y3_train,y4_train,y5_train]
test_y_sets = [y1_test,y2_test,y3_test,y4_test,y5_test]

### Logistic Regression

In [25]:
#creating a pipeline in order to grid search 
pipe = Pipeline([('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-8, 4, 11)},
                {'classifier': [LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-8, 4, 11)},
                {'classifier': [LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']}
                ]
# Create grid search 
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                   scoring=['accuracy', 'roc_auc', 'f1'], refit=False,
                   verbose=0)

In [22]:
#fitting the models with 5 fold cross validation
logreg_models = []

for i in range(5):
    logreg_models.append(clf.fit(train_X_sets[i],train_y_sets[i]))

In [26]:
#initializing array of training and test set predicted values
logreg_train_roc_pred = []
logreg_test_roc_pred = []

logreg_train_acc_pred = []
logreg_test_acc_pred = []

logreg_train_f1_pred = []
logreg_test_f1_pred = []

In [27]:
for i in range(5):
    proc = logreg_models[i].cv_results_['params'][ np.argmin(logreg_models[i].cv_results_['rank_test_roc_auc'])]
    pacc = logreg_models[i].cv_results_['params'][ np.argmin(logreg_models[i].cv_results_['rank_test_accuracy'])]
    pf1 = logreg_models[i].cv_results_['params'][ np.argmin(logreg_models[i].cv_results_['rank_test_f1'])]
    
    pipe.set_params(**proc)
    pipe.fit(train_X_sets[i],train_y_sets[i])

    logreg_train_roc_pred.append(pipe.predict(train_X_sets[i]))
    logreg_test_roc_pred.append(pipe.predict(test_X_sets[i]))
    
    pipe.set_params(**pacc)
    pipe.fit(train_X_sets[i],train_y_sets[i])
    
    logreg_train_acc_pred.append(pipe.predict(train_X_sets[i]))
    logreg_test_acc_pred.append(pipe.predict(test_X_sets[i]))
    
    pipe.set_params(**pf1)
    pipe.fit(train_X_sets[i],train_y_sets[i])
    
    logreg_train_f1_pred.append(pipe.predict(train_X_sets[i]))
    logreg_test_f1_pred.append(pipe.predict(test_X_sets[i]))

In [28]:
#initializing scores arrays for each metric, random forest on adult dataset
logreg_train_roc_scores = []
logreg_train_acc_scores = []
logreg_train_f1_scores = []
logreg_test_roc_scores = []
logreg_test_acc_scores = []
logreg_test_f1_scores = []

In [31]:
#populating the scores arrays 
for i in range(5):
    logreg_train_roc_scores.append(roc_auc_score(train_y_sets[i],logreg_train_roc_pred[i]))
    logreg_train_acc_scores.append(accuracy_score(train_y_sets[i],logreg_train_acc_pred[i]))
    logreg_train_f1_scores.append(f1_score(train_y_sets[i],logreg_train_f1_pred[i]))
    logreg_test_roc_scores.append(roc_auc_score(test_y_sets[i],logreg_test_roc_pred[i]))
    logreg_test_acc_scores.append(accuracy_score(test_y_sets[i],logreg_test_acc_pred[i]))
    logreg_test_f1_scores.append(f1_score(test_y_sets[i],logreg_test_f1_pred[i]))

In [32]:
#mean of each metric across trials
logreg_train_mean_roc = np.mean(logreg_train_roc_scores)
logreg_train_mean_acc = np.mean(logreg_train_acc_scores)
logreg_train_mean_f1 = np.mean(logreg_train_f1_scores)
logreg_test_mean_roc = np.mean(logreg_test_roc_scores)
logreg_test_mean_acc = np.mean(logreg_test_acc_scores)
logreg_test_mean_f1 = np.mean(logreg_test_f1_scores)

In [33]:
#mean across metrics 
logreg_metric_mean_test = np.mean([logreg_test_roc_scores,logreg_test_acc_scores,logreg_test_f1_scores])
logreg_metric_mean_train = np.mean([logreg_train_roc_scores,logreg_train_acc_scores,logreg_train_f1_scores])

### Random Forest

In [23]:
#creating a random forest object in order to grid search 
pipe2 =  RandomForestClassifier(criterion='entropy')

#setting the possible options for hyperparameters 
params = [{'n_estimators':[1024],'max_features':[1,2,4,6,8,12,16,20]}]

#creating a gridsearch object 
clf2 = GridSearchCV(pipe2, params, cv=StratifiedKFold(n_splits=5), scoring=['accuracy', 'roc_auc', 'f1'], refit=False, verbose=0)

In [24]:
#fitting the models with 5 fold cross validation
rf_models = []

for i in range(5):
    rf_models.append(clf2.fit(train_X_sets[i],train_y_sets[i]))

In [34]:
#initializing array of training and test set predicted values
rf_train_roc_pred = []
rf_test_roc_pred = []

rf_train_acc_pred = []
rf_test_acc_pred = []

rf_train_f1_pred = []
rf_test_f1_pred = []

In [35]:
for i in range(5):
    proc = rf_models[i].cv_results_['params'][ np.argmin(rf_models[i].cv_results_['rank_test_roc_auc'])]
    pacc = rf_models[i].cv_results_['params'][ np.argmin(rf_models[i].cv_results_['rank_test_accuracy'])]
    pf1 = rf_models[i].cv_results_['params'][ np.argmin(rf_models[i].cv_results_['rank_test_f1'])]
    
    pipe2.set_params(**proc)
    pipe2.fit(train_X_sets[i],train_y_sets[i])

    rf_train_roc_pred.append(pipe2.predict(train_X_sets[i]))
    rf_test_roc_pred.append(pipe2.predict(test_X_sets[i]))
    
    pipe2.set_params(**pacc)
    pipe2.fit(train_X_sets[i],train_y_sets[i])
    
    rf_train_acc_pred.append(pipe2.predict(train_X_sets[i]))
    rf_test_acc_pred.append(pipe2.predict(test_X_sets[i]))
    
    pipe2.set_params(**pf1)
    pipe2.fit(train_X_sets[i],train_y_sets[i])
    
    rf_train_f1_pred.append(pipe2.predict(train_X_sets[i]))
    rf_test_f1_pred.append(pipe2.predict(test_X_sets[i]))

In [36]:
#initializing scores arrays for each metric, random forest on adult dataset
rf_train_roc_scores = []
rf_train_acc_scores = []
rf_train_f1_scores = []
rf_test_roc_scores = []
rf_test_acc_scores = []
rf_test_f1_scores = []

In [37]:
#populating the scores arrays 
for i in range(5):
    rf_train_roc_scores.append(roc_auc_score(train_y_sets[i],rf_train_roc_pred[i]))
    rf_train_acc_scores.append(accuracy_score(train_y_sets[i],rf_train_acc_pred[i]))
    rf_train_f1_scores.append(f1_score(train_y_sets[i],rf_train_f1_pred[i]))
    rf_test_roc_scores.append(roc_auc_score(test_y_sets[i],rf_test_roc_pred[i]))
    rf_test_acc_scores.append(accuracy_score(test_y_sets[i],rf_test_acc_pred[i]))
    rf_test_f1_scores.append(f1_score(test_y_sets[i],rf_test_f1_pred[i]))

In [44]:
#mean of each metric across trials
rf_train_mean_roc = np.mean(rf_train_roc_scores)
rf_train_mean_acc = np.mean(rf_train_acc_scores)
rf_train_mean_f1 = np.mean(rf_train_f1_scores)
rf_test_mean_roc = np.mean(rf_test_roc_scores)
rf_test_mean_acc = np.mean(rf_test_acc_scores)
rf_test_mean_f1 = np.mean(rf_test_f1_scores)

In [45]:
#mean across metrics 
rf_metric_mean_test = np.mean([rf_test_roc_scores,rf_test_acc_scores,rf_test_f1_scores])
rf_metric_mean_train = np.mean([rf_train_roc_scores,rf_train_acc_scores,rf_train_f1_scores])

### kNN

In [19]:
#creating a k Nearest Neighbors object in order to grid search 
pipe3 =  KNeighborsClassifier()
step = 420/26
k = np.arange(1,420,step,dtype=int)
k_params = [{'n_neighbors':k,'weights':['uniform','distance'],'metric':['euclidean','manhattan']}]

clf3 = GridSearchCV(pipe3, k_params, cv=StratifiedKFold(n_splits=5), scoring=['accuracy', 'roc_auc', 'f1'], refit=False, verbose=0)

In [20]:
#fitting the models with 5 fold cross validation
knn_models = []

for i in range(5):
    knn_models.append(clf3.fit(train_X_sets[i],train_y_sets[i]))

In [40]:
#initializing array of training and test set predicted values
knn_train_roc_pred = []
knn_test_roc_pred = []

knn_train_acc_pred = []
knn_test_acc_pred = []

knn_train_f1_pred = []
knn_test_f1_pred = []

In [41]:
for i in range(5):
    proc = knn_models[i].cv_results_['params'][ np.argmin(knn_models[i].cv_results_['rank_test_roc_auc'])]
    pacc = knn_models[i].cv_results_['params'][ np.argmin(knn_models[i].cv_results_['rank_test_accuracy'])]
    pf1 = knn_models[i].cv_results_['params'][ np.argmin(knn_models[i].cv_results_['rank_test_f1'])]
    
    pipe3.set_params(**proc)
    pipe3.fit(train_X_sets[i],train_y_sets[i])

    knn_train_roc_pred.append(pipe3.predict(train_X_sets[i]))
    knn_test_roc_pred.append(pipe3.predict(test_X_sets[i]))
    
    pipe3.set_params(**pacc)
    pipe3.fit(train_X_sets[i],train_y_sets[i])
    
    knn_train_acc_pred.append(pipe3.predict(train_X_sets[i]))
    knn_test_acc_pred.append(pipe3.predict(test_X_sets[i]))
    
    pipe3.set_params(**pf1)
    pipe3.fit(train_X_sets[i],train_y_sets[i])
    
    knn_train_f1_pred.append(pipe3.predict(train_X_sets[i]))
    knn_test_f1_pred.append(pipe3.predict(test_X_sets[i]))

In [42]:
#initializing scores arrays for each metric, random forest on adult dataset
knn_train_roc_scores = []
knn_train_acc_scores = []
knn_train_f1_scores = []
knn_test_roc_scores = []
knn_test_acc_scores = []
knn_test_f1_scores = []

In [43]:
#populating the scores arrays 
for i in range(5):
    knn_train_roc_scores.append(roc_auc_score(train_y_sets[i],knn_train_roc_pred[i]))
    knn_train_acc_scores.append(accuracy_score(train_y_sets[i],knn_train_acc_pred[i]))
    knn_train_f1_scores.append(f1_score(train_y_sets[i],knn_train_f1_pred[i]))
    knn_test_roc_scores.append(roc_auc_score(test_y_sets[i],knn_test_roc_pred[i]))
    knn_test_acc_scores.append(accuracy_score(test_y_sets[i],knn_test_acc_pred[i]))
    knn_test_f1_scores.append(f1_score(test_y_sets[i],knn_test_f1_pred[i]))

In [46]:
#mean of each metric across trials
knn_train_mean_roc = np.mean(knn_train_roc_scores)
knn_train_mean_acc = np.mean(knn_train_acc_scores)
knn_train_mean_f1 = np.mean(knn_train_f1_scores)
knn_test_mean_roc = np.mean(knn_test_roc_scores)
knn_test_mean_acc = np.mean(knn_test_acc_scores)
knn_test_mean_f1 = np.mean(knn_test_f1_scores)

In [47]:
#mean across metrics 
knn_metric_mean_test = np.mean([knn_test_roc_scores,knn_test_acc_scores,knn_test_f1_scores])
knn_metric_mean_train = np.mean([knn_train_roc_scores,knn_train_acc_scores,knn_train_f1_scores])

### Calculations for ttest_rel

In [52]:
from scipy import stats

In [53]:
knn_mushroom_acc = knn_test_acc_scores
knn_mushroom_roc = knn_test_roc_scores
knn_mushroom_f1 = knn_test_f1_scores
lg_mushroom_acc = logreg_test_acc_scores
lg_mushroom_roc = logreg_test_roc_scores
lg_mushroom_f1 = logreg_test_f1_scores
rf_mushroom_acc = rf_test_acc_scores
rf_mushroom_roc = rf_test_roc_scores
rf_mushroom_f1 = rf_test_f1_scores



In [54]:
knn_adult_acc = [0.8332365747460088,0.8316037735849057,0.8384252539912918,0.8316400580551524,0.8354136429608128]
knn_adult_roc = [0.729340334922076,0.7126866812028831,0.7296946617794944,0.7135816695451274,0.7208615715683548]
knn_adult_f1 = [0.8936061854715496,0.8936599225534451,0.8972803395538742,0.8935193684597027,0.8959394356503786]
lg_adult_acc = [0.8471698113207548,0.8445936139332366, 0.8494920174165457,0.8473149492017417, 0.85]
lg_adult_roc = [0.7603485011157332,0.7496773540099201,0.753861142060206,0.7551723793723366,0.7646762156131413]
lg_adult_f1 = [0.9025107023024412,0.9013520049750098,0.9045515210087901,0.9029162052417865,0.904455948969215]
rf_adult_acc = [0.846988388969521,0.8545718432510885,0.8527939042089986,0.8533018867924528,0.8503628447024674]
rf_adult_roc = [0.766319170366713,0.7678401591361451,0.7674742882447559,0.767920867951701,0.7687750533472463]
rf_adult_f1 = [0.9026420726161882,0.9069082987648027, 0.9059467991272456, 0.9051465282465492, 0.9042543288028301] 

In [64]:
knn_energy_acc = [0.9619089316987741,0.9645359019264448,0.9658493870402802,0.9643169877408057,0.9621278458844134]
knn_energy_roc = [0.9604195433641599,0.9621458888559866,0.9638302910430611,0.9618298929730569,0.9583356493490452]
knn_energy_f1 = [0.9578079534432591,0.9614101953311099,0.9620991253644314,0.9616741123912533,0.9584034623707623]
lg_energy_acc = [0.9597197898423818,0.9590630472854641,0.9599387040280211,0.9575306479859895,0.9557793345008757]
lg_energy_roc = [0.9581214730238297,0.9582207457309248,0.9592430605370496,0.956646590492829,0.9542957459423405]
lg_energy_f1 = [0.9549019607843137,0.9551881140666187,0.9553984889105533,0.9540502131691142,0.9511841469308844]
rf_energy_acc = [0.9658493870402802,0.9691330998248686,0.9669439579684763,0.9660683012259195,0.9649737302977233]
rf_energy_roc = [0.9647741381511006,0.9678277070378049,0.9666812532548986,0.9677358068175321,0.9636771535858099]
rf_energy_f1 = [0.9612590799031476,0.9668089141773352,0.9633584081533608,0.9641097818437719,0.9616306954436451]

In [65]:
knn_room_acc = [0.9937803692905733,0.9933916423712342,0.9931972789115646,0.9920310981535472,0.9918367346938776]
knn_room_roc = [0.991076281494986,0.9895319132807763,0.9892142560822873,0.9914658607101671,0.9895326399278604]
knn_room_f1 = [0.9939671312669024,0.992190711056309,0.9927641099855281,0.9929401993355482,0.9930628547403826]
lg_room_acc = [0.991642371234208,0.9908649173955296,0.99067055393586,0.9922254616132167,0.9922254616132167]
lg_room_roc = [0.9917291217143159,0.9911590357792602,0.9910838032364212,0.992562118222742,0.9925409912570834]
lg_room_f1 = [0.9910732821258045,0.9903629280295263,0.9900990099009901,0.9917046868519287,0.9916177703269069]
rf_room_acc = [0.9926141885325559,0.9926141885325559,0.9922254616132167,0.9926141885325559,0.9914480077745383]
rf_room_roc =[0.9928147342697391,0.9930021983020012,0.9928095040548583,0.9925309202004688,0.9921185451558246]
rf_room_f1 = [0.9920767306088406,0.9921649484536083,0.9923411302007866,0.9925062447960034,0.991359325605901]

In [66]:
knn_acc_scores = knn_room_acc+knn_energy_acc+knn_adult_acc+knn_mushroom_acc
knn_roc_scores = knn_room_roc+knn_energy_roc+knn_adult_roc+knn_mushroom_roc
knn_f1_scores = knn_room_f1+knn_energy_f1+knn_adult_f1+knn_mushroom_f1
lg_acc_scores = lg_room_acc+lg_energy_acc+lg_adult_acc+lg_mushroom_acc
lg_roc_scores = lg_room_roc+lg_energy_roc+lg_adult_roc+lg_mushroom_roc
lg_f1_scores = lg_room_f1+lg_energy_f1+lg_adult_f1+lg_mushroom_f1
rf_acc_scores = rf_room_acc+rf_energy_acc+rf_adult_acc+rf_mushroom_acc
rf_roc_scores = rf_room_roc+rf_energy_roc+rf_adult_roc+rf_mushroom_roc
rf_f1_scores = rf_room_f1+rf_energy_f1+rf_adult_f1+rf_mushroom_f1



In [141]:
raw_adult_scores = {'knn_adult_acc':knn_adult_acc,
                   'knn_adult_roc':knn_adult_roc,
                   'knn_adult_f1':knn_adult_f1,
                   'rf_adult_acc':rf_adult_acc,
                   'rf_adult_roc':rf_adult_roc,
                   'rf_adult_f1':rf_adult_f1,
                   'lg_adult_acc':lg_adult_acc,
                   'lg_adult_roc':lg_adult_roc,
                   'lg_adult_f1':lg_adult_f1}

raw_mush_scores = {'knn_mushroom_acc':knn_mushroom_acc,
                   'knn_mushroom_roc':knn_mushroom_roc,
                   'knn_mushroom_f1':knn_mushroom_f1,
                   'rf_mushroom_acc':rf_mushroom_acc,
                   'rf_mushroom_roc':rf_mushroom_roc,
                   'rf_mushroom_f1':rf_mushroom_f1,
                   'lg_mushroom_acc':lg_mushroom_acc,
                   'lg_mushroom_roc':lg_mushroom_roc,
                   'lg_mushroom_f1':lg_mushroom_f1}

raw_ener_scores = {'knn_energy_acc':knn_energy_acc,
                   'knn_energy_roc':knn_energy_roc,
                   'knn_energy_f1':knn_energy_f1,
                   'rf_energy_acc':rf_energy_acc,
                   'rf_energy_roc':rf_energy_roc,
                   'rf_energy_f1':rf_energy_f1,
                   'lg_energy_acc':lg_energy_acc,
                   'lg_energy_roc':lg_energy_roc,
                   'lg_energy_f1':lg_energy_f1}                  

raw_room_scores = {'knn_room_acc':knn_room_acc,
                   'knn_room_roc':knn_room_roc,
                   'knn_room_f1':knn_room_f1,
                   'rf_room_acc':rf_room_acc,
                   'rf_room_roc':rf_room_roc,
                   'rf_room_f1':rf_room_f1,
                   'lg_room_acc':lg_room_acc,
                   'lg_room_roc':lg_room_roc,
                   'lg_room_f1':lg_room_f1,
                  }

In [146]:
df1 = pd.DataFrame(data=raw_adult_scores)

In [147]:
df2 = pd.DataFrame(data=raw_mush_scores)

In [148]:
df3 = pd.DataFrame(data=raw_ener_scores)

In [149]:
df4 = pd.DataFrame(data=raw_room_scores)

In [150]:
df1.to_csv(r'raw1.csv', index = False)
df2.to_csv(r'raw2.csv', index = False)
df3.to_csv(r'raw3.csv', index = False)
df4.to_csv(r'raw4.csv', index = False)

knn vs logreg in accuracy: 

In [89]:
knn_lg_acc = stats.ttest_rel(lg_acc_scores,knn_acc_scores)

knn vs logreg in roc auc:

In [90]:
knn_lg_roc = stats.ttest_rel(lg_roc_scores,knn_roc_scores)

knn vs logreg in f1:

In [91]:
knn_lg_f1 = stats.ttest_rel(lg_f1_scores,knn_f1_scores)

knn vs rf in accuracy:

In [92]:
knn_rf_acc = stats.ttest_rel(rf_acc_scores,knn_acc_scores)

knn vs rf in roc auc:

In [93]:
knn_rf_roc = stats.ttest_rel(rf_roc_scores,knn_roc_scores)

knn vs rf in f1:

In [94]:
knn_rf_f1 = stats.ttest_rel(rf_f1_scores,knn_f1_scores)

logreg vs rf in accuracy:

In [124]:
lg_rf_acc = stats.ttest_rel(rf_acc_scores,lg_acc_scores)

logreg vs rf in roc auc:

In [126]:
lg_rf_roc = stats.ttest_rel(rf_roc_scores,lg_roc_scores)

logreg vs rf in f1:

In [128]:
lg_rf_f1 = stats.ttest_rel(rf_f1_scores,lg_f1_scores)

knn adult vs rf adult:

In [120]:
knn_adult = knn_adult_roc + knn_adult_acc + knn_adult_f1

In [121]:
rf_adult = rf_adult_roc + rf_adult_acc + rf_adult_f1

In [123]:
stats.ttest_rel(rf_adult,knn_adult)

Ttest_relResult(statistic=5.6121065205010865, pvalue=6.407382622423274e-05)

In [157]:
pd.DataFrame(knn_models[0].cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,param_weights,params,split0_test_accuracy,split1_test_accuracy,...,std_test_roc_auc,rank_test_roc_auc,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,0.029883,0.001267,0.971169,0.032823,euclidean,1,uniform,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei...",1.000,1.000,...,0.000000e+00,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
1,0.028309,0.000131,0.948556,0.029314,euclidean,1,distance,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei...",1.000,1.000,...,0.000000e+00,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
2,0.028468,0.000210,1.259415,0.007080,euclidean,17,uniform,"{'metric': 'euclidean', 'n_neighbors': 17, 'we...",0.999,0.998,...,8.009259e-07,9,0.999034,0.998069,1.000000,0.999036,0.999036,0.999035,0.000610,12
3,0.028647,0.000333,1.236815,0.010192,euclidean,17,distance,"{'metric': 'euclidean', 'n_neighbors': 17, 'we...",0.999,0.999,...,0.000000e+00,1,0.999034,0.999034,1.000000,0.999036,1.000000,0.999421,0.000473,8
4,0.029429,0.001895,1.338305,0.006351,euclidean,33,uniform,"{'metric': 'euclidean', 'n_neighbors': 33, 'we...",0.999,0.997,...,4.268029e-05,16,0.999034,0.997107,1.000000,0.998073,0.997113,0.998265,0.001123,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0.028761,0.000258,1.578315,0.007186,manhattan,369,distance,"{'metric': 'manhattan', 'n_neighbors': 369, 'w...",0.985,0.980,...,1.555200e-03,61,0.985479,0.980769,0.986460,0.984526,0.970048,0.981457,0.006021,32
100,0.028810,0.000256,1.599161,0.004768,manhattan,385,uniform,"{'metric': 'manhattan', 'n_neighbors': 385, 'w...",0.929,0.897,...,3.187374e-03,101,0.934683,0.907954,0.914855,0.903342,0.894689,0.911105,0.013488,101
101,0.028884,0.000399,1.584158,0.004198,manhattan,385,distance,"{'metric': 'manhattan', 'n_neighbors': 385, 'w...",0.985,0.980,...,1.673020e-03,65,0.985479,0.980769,0.986460,0.986486,0.970048,0.981849,0.006267,30
102,0.028984,0.000211,1.613723,0.008429,manhattan,401,uniform,"{'metric': 'manhattan', 'n_neighbors': 401, 'w...",0.924,0.891,...,3.275880e-03,103,0.930403,0.903111,0.911051,0.907457,0.894265,0.909258,0.011968,103


In [160]:
knn_models[1].cv_results_['params'][ np.argmin(knn_models[i].cv_results_['rank_test_roc_auc'])]

{'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}

In [161]:
list(X)

['cap-shape_b',
 'cap-shape_c',
 'cap-shape_f',
 'cap-shape_k',
 'cap-shape_s',
 'cap-shape_x',
 'cap-surface_f',
 'cap-surface_g',
 'cap-surface_s',
 'cap-surface_y',
 'cap-color_b',
 'cap-color_c',
 'cap-color_e',
 'cap-color_g',
 'cap-color_n',
 'cap-color_p',
 'cap-color_r',
 'cap-color_u',
 'cap-color_w',
 'cap-color_y',
 'bruises_f',
 'bruises_t',
 'odor_a',
 'odor_c',
 'odor_f',
 'odor_l',
 'odor_m',
 'odor_n',
 'odor_p',
 'odor_s',
 'odor_y',
 'gill-attachment_a',
 'gill-attachment_f',
 'gill-spacing_c',
 'gill-spacing_w',
 'gill-size_b',
 'gill-size_n',
 'gill-color_b',
 'gill-color_e',
 'gill-color_g',
 'gill-color_h',
 'gill-color_k',
 'gill-color_n',
 'gill-color_o',
 'gill-color_p',
 'gill-color_r',
 'gill-color_u',
 'gill-color_w',
 'gill-color_y',
 'stalk-shape_e',
 'stalk-shape_t',
 'stalk-root_b',
 'stalk-root_c',
 'stalk-root_e',
 'stalk-root_r',
 'stalk-surface-above_f',
 'stalk-surface-above_k',
 'stalk-surface-above_s',
 'stalk-surface-above_y',
 'stalk-surface-bel